# Interoperability- Preparing the MuData dataset

In [3]:
import lamindb as ln

→ connected lamindb: theislab/sc-best-practices


In [4]:
assert ln.setup.settings.instance.slug == "theislab/sc-best-practices"

In [5]:
ln.track()

→ loaded Transform('sHhbAE1UThuC0000'), re-started Run('bn9MrSp1...') at 2025-03-20 17:16:57 UTC
→ notebook imports: anndata==0.11.3 lamindb==1.3.0 mudata==0.3.1 numpy==2.1.3


In [7]:
import mudata as md
import numpy as np
from anndata import AnnData
from mudata import MuData

md.set_options(pull_on_update=False)

# Use modern random number generation
rng = np.random.default_rng(seed=1)

# create random data
n, d, k = 1000, 100, 10
z = rng.normal(loc=np.arange(k), scale=np.arange(k) * 2, size=(n, k))
w = rng.normal(size=(d, k))
y = np.dot(z, w.T)

# create AnnData from the matrix
adata = AnnData(y)
adata.obs_names = [f"obs_{i + 1}" for i in range(n)]
adata.var_names = [f"var_{j + 1}" for j in range(d)]

if adata.obs.shape[1] == 0:
    adata.obs["dummy_obs"] = "placeholder"

if adata.var.shape[1] == 0:
    adata.var["dummy_var"] = "placeholder"

# second AnnData object
d2 = 50
w2 = rng.normal(size=(d2, k))
y2 = np.dot(z, w2.T)

adata2 = AnnData(y2)
adata2.obs_names = [f"obs_{i + 1}" for i in range(n)]
adata2.var_names = [f"var2_{j + 1}" for j in range(d2)]

if adata2.obs.shape[1] == 0:
    adata2.obs["dummy_obs"] = "placeholder"

if adata2.var.shape[1] == 0:
    adata2.var["dummy_var"] = "placeholder"

mdata = MuData({"A": adata, "B": adata2})
mdata.write("interoperability_mdata.h5mu")

In [8]:
af = ln.Artifact(
    "interoperability_mdata.h5mu",
    key="introduction/interoperability_mdata.h5mu",
    description="MuData object for interoperability chapter",
).save()
af

→ creating new artifact version for key='introduction/interoperability_mdata.h5mu' (storage: 's3://lamin-eu-central-1/VPwcjx3CDAa2')
... uploading interoperability_mdata.h5mu: 100.0%
! The cache path /Users/seohyon/Library/Caches/lamindb/lamin-eu-central-1/VPwcjx3CDAa2/introduction/interoperability_mdata.h5mu already exists, replacing it.


Artifact(uid='d0BIszdg2GPp3Nh80001', is_latest=True, key='introduction/interoperability_mdata.h5mu', description='MuData object for interoperability chapter', suffix='.h5mu', otype='MuData', size=1405984, hash='NX7ugtFo7KkZVUmwaASFiQ', space_id=1, storage_id=1, run_id=4, created_by_id=5, created_at=2025-03-20 17:29:41 UTC)